In [1]:
pip install datasets

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 80.3 MB/s 
     |████████████████████████████████| 136 kB 85.7 MB/s 
     |████████████████████████████████| 212 kB 86.4 MB/s 
     |████████████████████████████████| 77 kB 1.8 MB/s 
     |████████████████████████████████| 127 kB 79.0 MB/s 
     |████████████████████████████████| 271 kB 88.0 MB/s 
     |████████████████████████████████| 144 kB 84.4 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
pip install transformers


     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 68.3 MB/s 
     |████████████████████████████████| 596 kB 76.0 MB/s 
     |████████████████████████████████| 895 kB 69.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install tokenizers

In [4]:
pip install pysentimiento

     |████████████████████████████████| 312 kB 6.6 MB/s 
     |████████████████████████████████| 175 kB 81.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=aab57511039a21bf90a0e8658ccf4a04cbe2bb1a39ecbf9b145c175464eba778
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [5]:
from pysentimiento.preprocessing import preprocess_tweet

preprocess_tweet('RT @bellahtyrah: I love these tears😂😂😂TACHA OUR BITCOIN#GodMadeTacha' , lang='en')

'RT @USER: I love these tears emoji face with tears of joy emoji  emoji face with tears of joy emoji  emoji face with tears of joy emoji TACHA OUR BITCOIN#GodMadeTacha'

In [6]:
from datasets import load_dataset
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/tweet_train.csv',lineterminator='\n')

In [7]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(df['text']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample.replace('\n', '')
    sample = sample.replace('\r', '')
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we hit the 10K mark, save to file
        with open(f'/content/drive/MyDrive/text/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
 #after saving in 10K chunks, we will have ~3808 leftover samples, we save those now too
with open(f'/content/drive/MyDrive/text/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))


  0%|          | 0/2100000 [00:00<?, ?it/s]

In [8]:
from pathlib import Path
paths = [str(x) for x in Path('/content/drive/MyDrive/text').glob('**/*.txt')]


In [9]:
from tokenizers import ByteLevelBPETokenizer
# initialize
tokenizer = ByteLevelBPETokenizer()
# and train
tokenizer.train(files=paths[0:2], vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])


In [10]:
import os

os.mkdir('/content/drive/MyDrive/text/token')

tokenizer.save_model('/content/drive/MyDrive/text/token')

['/content/drive/MyDrive/text/token/vocab.json',
 '/content/drive/MyDrive/text/token/merges.txt']

In [11]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('/content/drive/MyDrive/text/token', max_len=512)

In [12]:
# Testing the tokenizer

test = 'RT @bellahtyrah: I love these tears😂😂😂TACHA OUR BITCOIN#GodMadeTacha'
tokenizer(test, max_length=15, padding='max_length', truncation=True)


{'input_ids': [0, 294, 264, 70, 11953, 449, 93, 1533, 30, 301, 1865, 1400, 20625, 5481, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
import torch

def mlm(tensor):
  rand = torch.rand(tensor.shape)
  mask_arr = (rand < 0.15) * (tensor > 2)
  for i in range(tensor.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    tensor[i,selection] = 4
  return(tensor)

In [14]:
from tqdm.auto import tqdm

input_ids = []
mask = []
labels = []

for path in tqdm(paths[:5]):
    with open(path, 'r', encoding='utf-8') as fp:
      lines = fp.read().split('\n')
    sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt') 
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids).clone())

  0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [16]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [18]:
dataset = Dataset(encodings)

In [19]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [20]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [21]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [23]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/3125 [00:00<?, ?it/s]